## Callback

`Callback` is a mechanism provided by `LangChain` that allows us to use hooks at various stages of an LLM application. This is useful for tasks such as logging, monitoring, and streaming. The execution logic for these tasks is defined by the `CallbackHandler`.

### Built-in Handlers

`StdOutCallbackHandler` is the most basic handler supported by LangChain. It prints all callback information to standard output, which is very useful for debugging.

In [1]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
chat_model = ChatOpenAI(temperature=0.5, model='gpt-3.5-turbo')
prompt = PromptTemplate.from_template("Who is {name}?")
chain = LLMChain(llm=chat_model, prompt=prompt, callbacks=[handler])
chain.run(name="Andrew Ng")



> Entering new LLMChain chain...
Prompt after formatting:
Who is Andrew Ng?

> Finished chain.


'Andrew Ng is a computer scientist and entrepreneur known for his work in artificial intelligence and machine learning. He co-founded Google Brain and is the co-founder and CEO of Landing AI. Ng is also a professor at Stanford University and has taught courses on machine learning and deep learning. He is a prominent figure in the field of AI and has made significant contributions to the development of AI technologies.'

## Custom Handler

We can implement a custom callback handler by inheriting from `BaseCallbackHandler`. Here is a simple example, `TimerHandler`, which will track the start and end times of Chain or LLM interactions and calculate the processing time for each interaction.

In [2]:
from langchain.callbacks.base import BaseCallbackHandler
import time

class TimeHandler(BaseCallbackHandler):

    def __init__(self) -> None:
        super().__init__()
        self.previous_ms = None
        self.durations = []

    def current_ms(self):
        return int(time.time() * 1000 + time.perf_counter() % 1 * 1000)

    def on_chain_start(self, serialized, inputs, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_chain_end(self, outputs, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration/1000)

    def on_llm_start(self, serialized, prompts, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_llm_end(self, response, **kwargs) -> None:
        if self.previous_ms:
          duration = self.current_ms() - self.previous_ms
          self.durations.append(duration/1000)


In [3]:
handler = TimeHandler()
chat_model = ChatOpenAI(temperature=0.5, model='gpt-3.5-turbo')
prompt = PromptTemplate.from_template("Where is {university}?")
chain = LLMChain(llm=chat_model, prompt=prompt, callbacks=[handler])
response = chain.run(university="USC")

print(response)
handler.durations

The University of Southern California (USC) is located in Los Angeles, California, United States.


[0.548]